In [1]:
# -*-encoding=utf-8-*-
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


In [2]:
# itemdata=pd.read_csv("C:\Users\scc\PycharmProjects\untitled1\datadig2\\tianchi_fresh_comp_train_item.csv")
# userdata=pd.read_csv("C:\Users\scc\PycharmProjects\untitled1\datadig2\\new_train_user.csv")
itemdata=pd.read_csv("D:\DAUM\\tianchi_fresh_comp_train_item.csv")
userdata=pd.read_csv("D:\DAUM\\new_train_user.csv")

In [3]:
#取出唯一的商品ID和类别
itemdata_id_category=itemdata.drop_duplicates(['item_id','item_category'])
print itemdata_id_category.shape

(422858, 3)


In [4]:
#取出有交互的商品数目
sub=pd.merge(itemdata_id_category,userdata,how='inner',on=['item_id','item_category'])
print sub.shape


(2084859, 7)


In [5]:
# print sub.head(10)
cols = list(sub)
print cols
cols=['user_id', 'item_id', 'item_geohash', 'item_category', 'behavior_type', 'date', 'hour']
df = sub.ix[:, cols]
del df['item_geohash']
print df.head(10)

['item_id', 'item_geohash', 'item_category', 'user_id', 'behavior_type', 'date', 'hour']


     user_id    item_id  item_category  behavior_type        date  hour
0   20723532  100002303           3368              1  2014-11-22    18
1   20723532  100002303           3368              1  2014-11-22    18
2   20723532  100002303           3368              1  2014-11-22    18
3   44416664  100002303           3368              1  2014-12-16    15
4   44416664  100002303           3368              1  2014-12-16    15
5  142281897  100002303           3368              1  2014-12-02    23
6  109084811  100003592           7995              1  2014-11-24    18
7  134325108  100006838          12630              1  2014-11-19    21
8    5015420  100008089           7791              1  2014-12-09    20
9  135142209  100012750           9614              1  2014-11-22    23


In [6]:
#选取16号作为特征数据
feature_date = '2014-12-16'
df3 = df[(df.date == feature_date) & (df.behavior_type == 3) & (df.hour >= 18)]
print df3.shape
print df3.head(10)


(808, 6)
         user_id    item_id  item_category  behavior_type        date  hour
165     47184224  100079162           4778              3  2014-12-16    18
186    114582878  100079162           4778              3  2014-12-16    22
881     39717215  100479772           4282              3  2014-12-16    20
4170    64559982  102385587          11991              3  2014-12-16    20
6248    64500498  103784652           7467              3  2014-12-16    20
21010  119838957  111296625          10431              3  2014-12-16    23
21795  105315608  111663276           3064              3  2014-12-16    22
26552   44299667  113997292           4830              3  2014-12-16    18
28461   64509513   11488409           4172              3  2014-12-16    23
40551   24979429  121067339           7850              3  2014-12-16    21


In [7]:
df4=df[(df.date == feature_date) & (df.behavior_type == 4) & (df.hour >= 18)]
print df4.shape
print df4.head(10)

(292, 6)
          user_id    item_id  item_category  behavior_type        date  hour
5726    111068056   10332364          12553              4  2014-12-16    18
32347    39717215  116855617           6648              4  2014-12-16    20
42172    51859808  122014872          10501              4  2014-12-16    20
45398   115939997  123405281          12553              4  2014-12-16    22
49096    60817805  125336077          12090              4  2014-12-16    18
78115   105731936  140098198           4722              4  2014-12-16    18
87538    63660779  144357343           6648              4  2014-12-16    21
103836  134143587  152725975            198              4  2014-12-16    18
110784  140015654  156189761           3723              4  2014-12-16    22
112354   54287375  156994015           3925              4  2014-12-16    21


In [8]:
#16日当天加购并且购买
addandbuy=pd.merge(df3,df4,how='inner',on=['user_id','item_id'])
addandbuy['addandbuy']=1
print addandbuy.shape
print addandbuy.head(10)

(164, 11)
     user_id    item_id  item_category_x  behavior_type_x      date_x  hour_x  \
0   60817805  125336077            12090                3  2014-12-16      23   
1  134143587  152725975              198                3  2014-12-16      18   
2  140015654  156189761             3723                3  2014-12-16      22   
3   54287375  156994015             3925                3  2014-12-16      21   
4  117257585   17200562            10431                3  2014-12-16      21   
5  141938543  178789856              438                3  2014-12-16      20   
6   23926348  192722116            13034                3  2014-12-16      21   
7   45499133  198176152             7393                3  2014-12-16      22   
8    4365869  214867102             1963                3  2014-12-16      19   
9  110330743  215866724              438                3  2014-12-16      23   

   item_category_y  behavior_type_y      date_y  hour_y  addandbuy  
0            12090           

In [9]:
#18点是否加购的特征加入df3
df_18_ifadd=pd.merge(df3,addandbuy,how='left',on=['user_id','item_id'])
df_18_ifadd=df_18_ifadd.fillna(0)
print df_18_ifadd.shape
print df_18_ifadd.head(1)

(929, 15)
    user_id    item_id  item_category  behavior_type        date  hour  \
0  47184224  100079162           4778              3  2014-12-16    18   

   item_category_x  behavior_type_x date_x  hour_x  item_category_y  \
0              0.0              0.0      0     0.0              0.0   

   behavior_type_y date_y  hour_y  addandbuy  
0              0.0      0     0.0        0.0  


In [12]:
# 浏览量统计
df_brows_count = df[(df.date == feature_date) & (df.behavior_type == 1)]
df_brows_count = df_brows_count.groupby(['user_id', 'item_id'], as_index=False) \
    .behavior_type.agg({'brows_count': np.size})
print df_brows_count.shape
print df_brows_count.head(10)

(30073, 3)
   user_id    item_id  brows_count
0    59436  184081436            4
1    61797   83261906            3
2   134211    6491625            2
3   134211   79679783            2
4   134211   96616269            2
5   134211  330807277            2
6   161481  299961071            1
7   182343   72718120            1
8   182343  112543556            2
9   182343  259931960            2


In [13]:
# 收藏量统计
df_collect_count = df[(df.date == feature_date) & (df.behavior_type == 2)]
df_collect_count = df_collect_count.groupby(['user_id', 'item_id'], as_index=False) \
    .behavior_type.agg({'collect_count': np.size})
print df_collect_count.shape
print df_collect_count.head(10)


(1103, 3)
   user_id    item_id  collect_count
0   404232  188280995              1
1   431616  134366643              1
2   431616  154990025              1
3   431616  329624631              1
4   487386  100835909              1
5   487386  104562108              1
6   487386  129607071              1
7   487386  175235179              1
8   487386  305245091              1
9   487386  333169427              1


In [14]:
# 加购量统计
df_add_count = df[(df.date == feature_date) & (df.behavior_type == 3)]
df_add_count = df_add_count.groupby(['user_id', 'item_id'], as_index=False) \
    .behavior_type.agg({'add_count': np.size})
print df_add_count.shape
print df_add_count.head(10)

(1713, 3)
   user_id    item_id  add_count
0   519693  218956054          1
1   519693  235772999          1
2   536394   34937426          1
3   536394  221204794          1
4  1051240  124928480          1
5  1096774   57739007          1
6  1462804   38600005          1
7  1462804  109002008          1
8  1462804  140857791          1
9  1462804  151259325          1


In [15]:
df_buy_count = df[(df.date == feature_date) & (df.behavior_type == 4)]
df_buy_count = df_buy_count.groupby(['user_id', 'item_id'], as_index=False) \
    .behavior_type.agg({'buy_count': np.size})
print df_buy_count.shape
print df_buy_count.head(10)

(756, 3)
   user_id    item_id  buy_count
0   487386  154082709          1
1   487386  175235179          1
2   511884   75771428          1
3  1134976  284916088          1
4  1172767  293099318          1
5  1287220  213454496          1
6  1591612  388604292          1
7  2201473  204806750          1
8  2333977   61988840          1
9  2549731  228919733          1


In [16]:
# 合并字段
tezhengdf = df3.loc[:, ['user_id', 'item_id','item_category', 'addandbuy']]
tmpdf1=pd.merge(df_brows_count,df_collect_count,how='outer',on=['user_id','item_id'])
tmpdf2=pd.merge(df_add_count,df_buy_count,how='outer',on=['user_id','item_id'])
tmpdf3=pd.merge(tmpdf1,tmpdf2,how='outer',on=['user_id','item_id'])
feature_df=pd.merge(tmpdf3,tezhengdf,how='outer',on=['user_id','item_id'])
feature_df=feature_df.fillna(0)
print feature_df.shape
print feature_df.head()


(30198, 8)
    user_id      item_id  brows_count  collect_count  add_count  buy_count  \
0   59436.0  184081436.0          4.0            0.0        0.0        0.0   
1   61797.0   83261906.0          3.0            0.0        0.0        0.0   
2  134211.0    6491625.0          2.0            0.0        0.0        0.0   
3  134211.0   79679783.0          2.0            0.0        0.0        0.0   
4  134211.0   96616269.0          2.0            0.0        0.0        0.0   

   item_category  addandbuy  
0            0.0        0.0  
1            0.0        0.0  
2            0.0        0.0  
3            0.0        0.0  
4            0.0        0.0  


In [18]:
# 购买占浏览量的比例
feature_df['buyofbrows'] = feature_df['buy_count']/feature_df['brows_count']
# 购买占收藏的比例
feature_df['buyofcollect'] = feature_df['buy_count']/feature_df['collect_count']
# 购买占加购的比例
feature_df['buyofadd'] = feature_df['buy_count']/feature_df['add_count']
print feature_df.head(20)


     user_id      item_id  brows_count  collect_count  add_count  buy_count  \
0    59436.0  184081436.0          4.0            0.0        0.0        0.0   
1    61797.0   83261906.0          3.0            0.0        0.0        0.0   
2   134211.0    6491625.0          2.0            0.0        0.0        0.0   
3   134211.0   79679783.0          2.0            0.0        0.0        0.0   
4   134211.0   96616269.0          2.0            0.0        0.0        0.0   
5   134211.0  330807277.0          2.0            0.0        0.0        0.0   
6   161481.0  299961071.0          1.0            0.0        0.0        0.0   
7   182343.0   72718120.0          1.0            0.0        0.0        0.0   
8   182343.0  112543556.0          2.0            0.0        0.0        0.0   
9   182343.0  259931960.0          2.0            0.0        0.0        0.0   
10  182343.0  264704619.0          1.0            0.0        0.0        0.0   
11  183987.0  124581566.0          2.0            0.

In [20]:
#提取特征最近1天（用户）的浏览数量
u_df1 = df[(df.date == feature_date) & (df.behavior_type == 1)]
u_df1 = u_df1.groupby(['user_id'], as_index=False)['behavior_type'].agg({'u_brows_count': np.size})
print u_df1.head()




   user_id  u_brows_count
0    59436              4
1    61797              3
2   134211              8
3   161481              1
4   182343              6


In [21]:
#提取特征最近1天（用户）的收藏的记录数量
u_df2=df[(df.date==feature_date) & (df.behavior_type==2)]
u_df2 =u_df2.groupby(['user_id'], as_index=False)['behavior_type'].agg({'u_collect_count': np.size})
print u_df2.head()

   user_id  u_collect_count
0   404232                1
1   431616                3
2   487386                7
3   595845                2
4  1096774                1


In [22]:
#提取特征最近1天（用户）的加购的记录数量
u_df3=df[(df.date==feature_date) & (df.behavior_type==3)]
u_df3 =df3.groupby(['user_id'], as_index=False)['user_id'].agg({'u_add_count': np.size})
print u_df3.head()

   user_id  u_add_count
0  1051240            1
1  1096774            1
2  1462804            7
3  1591612            1
4  1903255            1


In [23]:
#提取特征最近1天（用户）的购买的记录数量
u_df4=df[(df.date==feature_date) & (df.behavior_type==4)]
u_df4 =df4.groupby(['user_id'], as_index=False)['user_id'].agg({'u_buy_count': np.size})
print u_df4.head()

   user_id  u_buy_count
0   487386            1
1  1287220            1
2  1591612            1
3  2201473            1
4  2333977            1


In [24]:
#将用户特征进行合并
u_feature_df=pd.merge(u_df1,u_df2,how='outer',on=['user_id'])
u_feature_df1=pd.merge(u_df3,u_df4,how='outer',on=['user_id'])
u_feature_df=pd.merge(u_feature_df,u_feature_df1,how='outer',on=['user_id'])
u_feature_df=u_feature_df.fillna(0)
print u_feature_df.shape,u_feature_df.head()

(4105, 5)     user_id  u_brows_count  u_collect_count  u_add_count  u_buy_count
0   59436.0            4.0              0.0          0.0          0.0
1   61797.0            3.0              0.0          0.0          0.0
2  134211.0            8.0              0.0          0.0          0.0
3  161481.0            1.0              0.0          0.0          0.0
4  182343.0            6.0              0.0          0.0          0.0


In [25]:
#将特征合并
feature_df=pd.merge(feature_df,u_feature_df,how='left',on=['user_id'])
feature_df=feature_df.fillna(0)
print feature_df.shape,feature_df.head(20)

(30198, 15)      user_id      item_id  brows_count  collect_count  add_count  buy_count  \
0    59436.0  184081436.0          4.0            0.0        0.0        0.0   
1    61797.0   83261906.0          3.0            0.0        0.0        0.0   
2   134211.0    6491625.0          2.0            0.0        0.0        0.0   
3   134211.0   79679783.0          2.0            0.0        0.0        0.0   
4   134211.0   96616269.0          2.0            0.0        0.0        0.0   
5   134211.0  330807277.0          2.0            0.0        0.0        0.0   
6   161481.0  299961071.0          1.0            0.0        0.0        0.0   
7   182343.0   72718120.0          1.0            0.0        0.0        0.0   
8   182343.0  112543556.0          2.0            0.0        0.0        0.0   
9   182343.0  259931960.0          2.0            0.0        0.0        0.0   
10  182343.0  264704619.0          1.0            0.0        0.0        0.0   
11  183987.0  124581566.0          2.0  

In [26]:
#取17号数据
target_df = df[(df.date == '2014-12-17') & (df.behavior_type == 4)]
target_df['target']=1
target_df=target_df.loc[:,['user_id','item_id','target']]
print target_df.shape
print target_df.head(20)


(720, 3)
         user_id    item_id  target
1025    42369712  100570822       1
4147   113190941  102360256       1
5200    33541423  102984080       1
8711   137607597  104883287       1
14989   34714627  108275033       1
17755  129098352  109691887       1
18744   62871584  110238507       1
18979  106342022  110310343       1
23441  131157314  112589217       1
29841   37165541   11572504       1
30874  102105235  116247903       1
33351  121875002  117300815       1
38776   28674134  120163823       1
40768   29120381  121174040       1
51726   16334101  126492618       1
52427  121299887  126777986       1
54373  137358666  127431729       1
54585  119279535  127517940       1
57431   10874175  129098469       1
58409   45470234  129662365       1


D:\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [27]:
feature_data=pd.merge(feature_df,target_df,how='left',on=['user_id','item_id']).drop_duplicates(['user_id','item_id'])
print feature_data.shape
feature_data=feature_data.fillna(0)
print feature_data.head(10)

(30183, 16)
    user_id      item_id  brows_count  collect_count  add_count  buy_count  \
0   59436.0  184081436.0          4.0            0.0        0.0        0.0   
1   61797.0   83261906.0          3.0            0.0        0.0        0.0   
2  134211.0    6491625.0          2.0            0.0        0.0        0.0   
3  134211.0   79679783.0          2.0            0.0        0.0        0.0   
4  134211.0   96616269.0          2.0            0.0        0.0        0.0   
5  134211.0  330807277.0          2.0            0.0        0.0        0.0   
6  161481.0  299961071.0          1.0            0.0        0.0        0.0   
7  182343.0   72718120.0          1.0            0.0        0.0        0.0   
8  182343.0  112543556.0          2.0            0.0        0.0        0.0   
9  182343.0  259931960.0          2.0            0.0        0.0        0.0   

   item_category  addandbuy  buyofbrows  buyofcollect  buyofadd  \
0            0.0        0.0         0.0           0.0       0.

In [28]:
positive_df=feature_data[feature_data['target']==1]
negative_df=feature_data[feature_data['target']==0]
negative_df=pd.DataFrame.sample(negative_df,positive_df.shape[0]*5)

In [29]:
print positive_df.shape
print negative_df.shape

(129, 16)
(645, 16)


In [32]:
#合并
sample=pd.concat([positive_df,negative_df],axis=0)
sample=pd.DataFrame.round(sample,2)
print sample.head(20)

         user_id      item_id  brows_count  collect_count  add_count  \
135    1096774.0   57739007.0          4.0            0.0        1.0   
322    1694224.0  340480775.0          3.0            0.0        0.0   
585    2630830.0  202530287.0          2.0            0.0        1.0   
887    3376717.0  288552625.0          4.0            0.0        1.0   
990    3687502.0  121555932.0          3.0            0.0        0.0   
1036   4041047.0   98723312.0          9.0            0.0        1.0   
1039   4041047.0  218700963.0          4.0            0.0        0.0   
1181   4564376.0  132929824.0          3.0            0.0        0.0   
1182   4564376.0  389342445.0          2.0            0.0        0.0   
1321   5352155.0  335956990.0          7.0            1.0        0.0   
1908  10874175.0  129098469.0          1.0            1.0        0.0   
2876  13698828.0  236600747.0         12.0            1.0        0.0   
2887  13698828.0  332555164.0          6.0            0.0       

In [33]:
sample.to_csv('D:\DAUM\\train.csv',index=False)